<a href="https://colab.research.google.com/github/Thanyarat281-6/dataviz2025/blob/main/2816_health_warehouse_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Lab: สร้าง Health Data Warehouse (ฉบับนักศึกษา)

**ชื่อ**:นางสาวธัญยรัตน์ สักการี **รหัสนักศึกษา**: 663020281-6

## 🎯 วัตถุประสงค์:
ฝึกปฏิบัติการสร้าง Data Warehouse จากฐานข้อมูลระบบเฝ้าระวังโรค NCDs

## 📚 สิ่งที่จะได้เรียนรู้:
1. การเชื่อมต่อและดึงข้อมูลจาก SQLite Database
2. การทำ Data Transformation (JOIN, Groupby, Merge)
3. การสร้าง Data Warehouse 5 ตาราง
4. การอัปโหลดข้อมูลไปยัง Google Sheets

## ⏱️ เวลาที่ใช้: 120 นาที

---

## 📦 Part 1: เตรียมสภาพแวดล้อม (10 นาที)

### Exercise 1.1: ติดตั้ง Libraries

**โจทย์**: ติดตั้ง packages ที่จำเป็นสำหรับ Lab นี้

**Hint**: ต้องติดตั้ง `gspread`, `google-auth`, `google-auth-oauthlib`, `google-auth-httplib2`, `openpyxl`

In [7]:
# TODO: ติดตั้ง packages ที่จำเป็น
# Hint: ใช้ !pip install ... -q

# เขียนโค้ดที่นี่
!pip install gspread google-auth google-auth-oauthlib google-auth-httplib2 openpyxl -q


print("✅ ติดตั้ง libraries สำเร็จ")

✅ ติดตั้ง libraries สำเร็จ


### Exercise 1.2: Mount Google Drive

**โจทย์**: Mount Google Drive และตรวจสอบว่ามีไฟล์ที่ต้องการหรือไม่

**ไฟล์ที่ต้องการ**:
- `ncds_sample.db`
- `districts_with_zone_20250930_084409.xlsx`

**Path**: `/content/drive/MyDrive/Dashboard-Teaching-2025/`

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# TODO: Mount Google Drive
# Hint: ใช้ from google.colab import drive
#       drive.mount('/content/drive')

# เขียนโค้ดที่นี่

from google.colab import drive
drive.mount('/content/drive')

# TODO: กำหนด path ของไฟล์
import os

SHARED_FOLDER = '/content/drive/MyDrive/Python&datavizz2025'
DB_PATH = f'{SHARED_FOLDER}/ncds_sample.db'
DISTRICTS_PATH = f'{SHARED_FOLDER}/districts_with_zone_20250930_084409.xlsx'

print("✅ Mount Google Drive สำเร็จ")

# TODO: ตรวจสอบว่าไฟล์มีอยู่จริง
# Hint: ใช้ os.path.exists()

# เขียนโค้ดที่นี่

if os.path.exists(DB_PATH):
    print(f"✅ ไฟล์ Database '{os.path.basename(DB_PATH)}' พบแล้ว")
else:
    print(f"❌ ไม่พบไฟล์ Database ที่ '{DB_PATH}' โปรดตรวจสอบเส้นทาง")

if os.path.exists(DISTRICTS_PATH):
    print(f"✅ ไฟล์ Excel '{os.path.basename(DISTRICTS_PATH)}' พบแล้ว")
else:
    print(f"❌ ไม่พบไฟล์ Excel ที่ '{DISTRICTS_PATH}' โปรดตรวจสอบเส้นทาง")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Mount Google Drive สำเร็จ
✅ ไฟล์ Database 'ncds_sample.db' พบแล้ว
✅ ไฟล์ Excel 'districts_with_zone_20250930_084409.xlsx' พบแล้ว


### Exercise 1.3: Import Libraries

**โจทย์**: Import libraries ที่จำเป็นทั้งหมด

In [10]:
# TODO: Import libraries ที่จำเป็น
# Hint: sqlite3, pandas, numpy, datetime, gspread, warnings

# เขียนโค้ดที่นี่

import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
import gspread
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("✅ Import libraries สำเร็จ")

✅ Import libraries สำเร็จ


---

## 🔌 Part 2: เชื่อมต่อและดึงข้อมูล (15 นาที)

### Exercise 2.1: เชื่อมต่อ SQLite Database

**โจทย์**: เชื่อมต่อกับ `ncds_sample.db` และแสดงรายชื่อตารางทั้งหมดพร้อมจำนวนแถว

In [19]:
# TODO: เชื่อมต่อ SQLite Database
# Hint: ใช้ sqlite3.connect(DB_PATH)

# เขียนโค้ดที่นี่
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()


# TODO: ดูรายชื่อตารางทั้งหมดพร้อมจำนวนแถว
# Hint: SELECT name FROM sqlite_master WHERE type='table'
#       จากนั้น SELECT COUNT(*) FROM {table_name}

# เขียนโค้ดที่นี่
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()


### Exercise 2.2: ดึงข้อมูลจากตาราง

**โจทย์**: ดึงข้อมูลจาก 5 ตารางหลัก:
1. `ncds_profile`
2. `ncds_record`
3. `ncds_habit`
4. `campaign`
5. `memberdata`

**Hint**: ใช้ `pd.read_sql_query("SELECT * FROM table_name", conn)`

In [17]:
# TODO: ดึงข้อมูลจากแต่ละตาราง



print("📥 กำลังดึงข้อมูลจาก Database...")

# 1. ncds_profile
ncds_profile = pd.read_sql_query("SELECT * FROM ncds_profile", conn)
print(f"✅ ncds_profile: {len(ncds_profile)} แถว")

# 2. ncds_record
ncds_record = pd.read_sql_query("SELECT * FROM ncds_record", conn)
print(f"✅ ncds_record: {len(ncds_record)} แถว")

# 3. ncds_habit
ncds_habit =  pd.read_sql_query("SELECT * FROM ncds_habit", conn)
print(f"✅ ncds_habit: {len(ncds_habit)} แถว")

# 4. campaign
campaign =  pd.read_sql_query("SELECT * FROM campaign", conn)
print(f"✅ campaign: {len(campaign)} แถว")

# 5. memberdata
memberdata =  pd.read_sql_query("SELECT * FROM memberdata", conn)
print(f"✅ memberdata: {len(memberdata)} แถว")

📥 กำลังดึงข้อมูลจาก Database...
✅ ncds_profile: 192 แถว
✅ ncds_record: 192 แถว
✅ ncds_habit: 192 แถว
✅ campaign: 20 แถว
✅ memberdata: 1920 แถว


### Exercise 2.3: โหลดข้อมูลพื้นที่

**โจทย์**: โหลดข้อมูลพื้นที่จากไฟล์ Excel และแปลง `district_code` เป็น string

In [18]:
# TODO: โหลดข้อมูลพื้นที่
# Hint: ใช้ pd.read_excel()

districts_df = pd.read_excel(DISTRICTS_PATH)
districts_df['district_code'] =districts_df['district_code'].astype(str)

print(f"✅ โหลดข้อมูลพื้นที่: {len(districts_df)} แถว")
districts_df.head()

✅ โหลดข้อมูลพื้นที่: 7356 แถว


,district_code,district_name,amphur_code,amphur_name,province_code,province_name,lat,lng,zone
0,100101,แขวง พระบรมมหาราชวัง,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.751,100.492,4.0
1,100102,แขวง วังบูรพาภิรมย์,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.744,100.499,4.0
2,100103,แขวง วัดราชบพิธ,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.750,100.499,4.0
3,100104,แขวง สำราญราษฎร์,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.751,100.503,4.0
4,100105,แขวง ศาลเจ้าพ่อเสือ,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.754,100.497,4.0


---

## 🏗️ Part 3: สร้าง Data Warehouse (40 นาที)

### Exercise 3.1: Health Profile Warehouse

**โจทย์**: สร้างตาราง Health Profile Warehouse ตามขั้นตอนต่อไปนี้:

**ขั้นตอน**:
1. ดึงข้อมูล**ล่าสุด**ของแต่ละสมาชิกจาก `ncds_profile` และ `ncds_record`
2. นับจำนวนครั้งการตรวจของแต่ละสมาชิก
3. JOIN กับ `campaign` เพื่อได้ `district_code`
4. JOIN กับ `districts_df` เพื่อได้ข้อมูลพื้นที่
5. สร้าง DataFrame ตามคอลัมน์ที่กำหนด

**คอลัมน์ที่ต้องมี**:
- member_id
- จำนวนครั้งการตรวจ
- เพศ (แปลง 0→หญิง, 1→ชาย)
- อายุ
- ตำบล_id, ตำบล
- อำเภอ_id, อำเภอ
- จังหวัด_id, จังหวัด
- เขต
- กลุ่มอายุ
- ประวัติการเจ็บป่วย (code และ label)
- ประวัติแพ้ยา (code และ label)
- ประวัติแพ้อาหาร (code และ label)
- ญาติสายตรงNCDs (code และ label)

In [20]:
# Step 1: ดึงข้อมูลล่าสุดของแต่ละสมาชิก
# Hint: ใช้ .sort_values('dateadd').groupby('memberid').tail(1)

latest_profiles = ncds_profile.sort_values('dateadd').groupby('memberid').tail(1)
latest_records = ncds_record.sort_values('dateadd').groupby('memberid').tail(1)

print(f"✓ Latest profiles: {len(latest_profiles)} แถว")
print(f"✓ Latest records: {len(latest_records)} แถว")

✓ Latest profiles: 100 แถว
✓ Latest records: 100 แถว


In [21]:
# Step 2: นับจำนวนครั้งการตรวจ
# Hint: ใช้ .groupby('memberid').size().reset_index(name='จำนวนครั้งการตรวจ')

screening_count = ncds_record.groupby('memberid').size().reset_index(name='จำนวนครั้งการตรวจ')

print(f"✓ Screening count: {len(screening_count)} สมาชิก")
screening_count.head()

✓ Screening count: 100 สมาชิก


,memberid,จำนวนครั้งการตรวจ
0,10000000,3
1,10000001,3
2,10000002,3
3,10000003,2
4,10000004,2


In [23]:
# Step 3: JOIN กับ campaign
# Hint: ใช้ pd.merge() โดย left_on='campaignId', right_on='id'

campaign['district_code'] = campaign['district_code'].astype(str)

profile_campaign = pd.merge(latest_profiles, campaign, left_on='campaignId', right_on='id', how='left')

print(f"✓ Profile + Campaign: {len(profile_campaign)} แถว")

✓ Profile + Campaign: 100 แถว


In [24]:
# Step 4: JOIN กับ districts
# Hint: ใช้ pd.merge() โดย on='district_code'

profile_location = pd.merge(profile_campaign, districts_df, on='district_code', how='left')

print(f"✓ Profile + Location: {len(profile_location)} แถว")

✓ Profile + Location: 100 แถว


In [25]:
# Step 5: JOIN กับ record และ screening_count
# Hint: JOIN 2 ครั้ง - ครั้งแรกกับ latest_records, ครั้งที่สองกับ screening_count

profile_record = pd.merge(profile_location, latest_records, on='memberid', how='left')

final_data = pd.merge(profile_record, screening_count, on='memberid', how='left')

# แปลง NaN ในจำนวนครั้งการตรวจเป็น 0
final_data['จำนวนครั้งการตรวจ'] = final_data['จำนวนครั้งการตรวจ'].fillna(0).astype(int)

print(f"✓ Final data: {len(final_data)} แถว")
final_data.head()

✓ Final data: 100 แถว


,id_x,dateadd_x,detail,memberid,age,gender,campaignId,risk_bmi,risk_habit,risk_waistline,risk_bloodpressure,risk_fbs,risk_movement,risk_sleep,risk_2q,risk_stress,risk_vegetable,risk_sugar,risk_fat,risk_sodium,id_y,companyid,name,caption,datestart,datestop,image,icon,day,price,pay,promocode,memberchatbutton,lat_x,lng_x,address,address_name,address_note,postcode,appointment_online,district_code,district_name,amphur_code,amphur_name,province_code,province_name,lat_y,lng_y,zone,id,dateadd_y,drug_allergy,drug_allergy_note,food_allergy,food_allergy_note,record_coronary_artery_disease,record_emphysema,record_ncds,record_obesity,relatives_chronic_kidney_disease,relatives_emphysema,relatives_ncds,campaignid,record_cancer,record_chronic_kidney_disease,record_cirrhosis,record_dementia,record_diabetes,record_hyperlipidemia,record_hypertension,record_stroke,relatives_cancer,relatives_cirrhosis,relatives_coronary_artery_disease,relatives_dementia,relatives_diabetes,relatives_hyperlipidemia,relatives_hypertension,relatives_stroke,relatives_obesity,จำนวนครั้งการตรวจ
0,77,2025-06-02 00:00:00,0b654fa141f37aee35abf9d3555527b94bf7b9b2e167dc...,10000036,50,1,100008,low,medium,medium,high,,high,high,,medium,high,high,,medium,100008,100001,โครงการคัดกรองสุขภาพ (สำนักงานสาธารณสุขอำเภอเช...,โครงการคัดกรองสุขภาพประจำปี,2025-02-23,2025-12-31,,,36525,0.0,0,,"[""wound"",""patientprofile"",""sugar"",""bloodpressu...",18.801164,99.023845,ถนนสุขภาพดี อำเภอเชียงใหม่,สำนักงานสาธารณสุขอำเภอเชียงใหม่,,50000,1,500101,ต. ศรีภูมิ,5001.0,อ. เมืองเชียงใหม่,50.0,จ. เชียงใหม่,18.796,98.985,2.0,77,2025-06-02 00:00:00,1,ยาปฏิชีวนะ,-1,,0,0,-1,0,0,0,-1,100008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1
1,114,2025-06-10 00:00:00,da3b431e84ad23aecc05a3a6ca86c0aa3ee563a21e6b23...,10000058,44,0,100006,high,medium,low,high,low,low,low,,low,low,medium,medium,high,100006,100001,กลุ่มผู้ป่วยเบาหวาน (สำนักงานสาธารณสุขอำเภอนนท...,กลุ่มผู้ป่วยเบาหวาน,2025-01-25,2025-12-31,,,36525,0.0,0,,"[""wound"",""patientprofile"",""sugar"",""bloodpressu...",13.819272,100.558949,ถนนสุขภาพดี อำเภอนนทบุรี,สำนักงานสาธารณสุขอำเภอนนทบุรี,,11000,1,120102,ต. ตลาดขวัญ,1201.0,อ. เมืองนนทบุรี,12.0,จ. นนทบุรี,13.850,100.509,6.0,114,2025-06-10 00:00:00,-1,,-1,,0,0,-1,0,0,0,-1,100006,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2
2,54,2025-06-12 00:00:00,ffd144bbca0915dedc29ff24a4990a2a4a08ed615f3b98...,10000025,73,0,100015,high,medium,high,low,medium,medium,low,high,medium,medium,medium,low,low,100015,100001,โครงการคัดกรองสุขภาพ (สำนักงานสาธารณสุขอำเภอขอ...,โครงการคัดกรองสุขภาพประจำปี,2025-01-14,2025-12-31,,,36525,0.0,0,,"[""wound"",""patientprofile"",""sugar"",""bloodpressu...",16.447744,102.813163,ถนนสุขภาพดี อำเภอขอนแก่น,สำนักงานสาธารณสุขอำเภอขอนแก่น,,40000,1,400101,ต. ในเมือง,4001.0,อ. เมืองขอนแก่น,40.0,จ. ขอนแก่น,16.438,102.837,1.0,54,2025-06-12 00:00:00,1,เพนนิซิลลิน,-1,,0,0,-1,1,1,0,-1,100015,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1
3,82,2025-06-12 00:00:00,a33f5d57dd594b547ab361798301edb6a7075de5f9496a...,10000040,48,0,100008,low,low,,low,high,high,high,low,low,high,medium,high,high,100008,100001,โครงการคัดกรองสุขภาพ (สำนักงานสาธารณสุขอำเภอเช...,โครงการคัดกรองสุขภาพประจำปี,2025-02-23,2025-12-31,,,36525,0.0,0,,"[""wound"",""patientprofile"",""sugar"",""bloodpressu...",18.801164,99.023845,ถนนสุขภาพดี อำเภอเชียงใหม่,สำนักงานสาธารณสุขอำเภอเชียงใหม่,,50000,1,500101,ต. ศรีภูมิ,5001.0,อ. เมืองเชียงใหม่,50.0,จ. เชียงใหม่,18.796,98.985,2.0,82,2025-06-12 00:00:00,-1,,-1,,0,0,-1,1,0,0,1,100008,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1
4,142,2025-06-13 00:00:00,2ccc87026f02cff661c222f46039fb18cbc46d0900fce4...,10000076,62,0,100001,medium,low,medium,low,high,,low,medium,high,,low,high,,100001,100001,กลุ่มผู้ป่วยเบาหวาน (สำนักงานสาธารณสุขอำเภอกรุ...,กลุ่มผู้ป่วยเบาหวาน,2025-02-17,2025-12-31,,,36525,0.0,0,,"[""wound"",""patientprofile"",""sugar"",""bloodpressu...",13.733803,100.474121,ถนนสุขภาพดี อำเภอกรุงเทพมหา

In [27]:
# Step 6: สร้าง Helper Functions
# TODO: เขียนฟังก์ชันแปลงค่าต่างๆ

def get_gender_label(gender):
    """แปลงเพศ: 0→หญิง, 1→ชาย, อื่นๆ→999"""
    # เขียนโค้ดที่นี่
    if gender == 0:
        return 'หญิง'
    elif gender == 1:
        return 'ชาย'
    else:
        return 999

def get_age_group(age):
    """
    แปลงกลุ่มอายุ:
    - < 20: 'อายุต่ำกว่า 20 ปี'
    - 20-34: 'อายุระหว่าง 20-34 ปี'
    - 35-59: 'อายุระหว่าง 35-59 ปี'
    - >= 60: 'อายุ 60 ปีขึ้นไป'
    """
    # เขียนโค้ดที่นี่
    if age < 20:
        return 'อายุต่ำกว่า 20 ปี'
    elif 20 <= age <= 34:
        return 'อายุระหว่าง 20-34 ปี'
    elif 35 <= age <= 59:
        return 'อายุระหว่าง 35-59 ปี'
    elif age >= 60:
        return 'อายุ 60 ปีขึ้นไป'
    else:
        return 'ไม่ทราบ'


def convert_illness_history(value):
    """
    แปลงประวัติการเจ็บป่วย:
    - -1: 'ไม่มีประวัติ'
    - 1: 'มีประวัติการเจ็บป่วยรวม'
    - อื่นๆ: 'ไม่ทราบ'
    """
    # เขียนโค้ดที่นี่
    if value == -1:
        return 'ไม่มีประวัติ'
    elif value == 1:
        return 'มีประวัติการเจ็บป่วยรวม'
    else:
        return 'ไม่ทราบ'


# TODO: เขียนฟังก์ชันเพิ่มเติมสำหรับ drug_allergy, food_allergy, relatives_ncds
# (รูปแบบเหมือน convert_illness_history)

def convert_drug_allergy(value):
    """แปลงประวัติแพ้ยา: -1→ไม่มีประวัติ, 1→มีประวัติ, อื่นๆ→ไม่ทราบ"""
    if value == -1:
        return 'ไม่มีประวัติ'
    elif value == 1:
        return 'มีประวัติ'
    else:
        return 'ไม่ทราบ'

def convert_food_allergy(value):
    """แปลงประวัติแพ้อาหาร: -1→ไม่มีประวัติ, 1→มีประวัติ, อื่นๆ→ไม่ทราบ"""
    if value == -1:
        return 'ไม่มีประวัติ'
    elif value == 1:
        return 'มีประวัติ'
    else:
        return 'ไม่ทราบ'

def convert_relatives_ncds(value):
    """แปลงประวัติญาติสายตรงNCDs: -1→ไม่มีประวัติ, 1→มีประวัติ, อื่นๆ→ไม่ทราบ"""
    if value == -1:
        return 'ไม่มีประวัติ'
    elif value == 1:
        return 'มีประวัติ'
    else:
        return 'ไม่ทราบ'

In [28]:
# Step 7: สร้าง Health Profile Warehouse
# TODO: สร้าง DataFrame ตามคอลัมน์ที่กำหนด

health_profile_wh = pd.DataFrame({
    'member_id': final_data['memberid'],
    'จำนวนครั้งการตรวจ': final_data['จำนวนครั้งการตรวจ'],
    'เพศ': final_data['gender'].apply(get_gender_label),
    'อายุ': final_data['age'],
    'ตำบล_id': final_data['district_code'],
    'ตำบล': final_data['district_name'],
    'อำเภอ_id': final_data['amphur_code'],
    'อำเภอ': final_data['amphur_name'],
    'จังหวัด_id': final_data['province_code'],
    'จังหวัด': final_data['province_name'],
    'เขต': final_data['zone'],
    'กลุ่มอายุ': final_data['age'].apply(get_age_group),
    'ประวัติการเจ็บป่วย_code': final_data['record_ncds'],
    'ประวัติการเจ็บป่วย': final_data['record_ncds'].apply(convert_illness_history),
    'ประวัติแพ้ยา_code': final_data['drug_allergy'],
    'ประวัติแพ้ยา': final_data['drug_allergy'].apply(convert_drug_allergy),
    'ประวัติแพ้อาหาร_code': final_data['food_allergy'],
    'ประวัติแพ้อาหาร': final_data['food_allergy'].apply(convert_food_allergy),
    'ญาติสายตรงNCDs_code': final_data['relatives_ncds'],
    'ญาติสายตรงNCDs': final_data['relatives_ncds'].apply(convert_relatives_ncds)
})

print(f"✅ Health Profile Warehouse: {len(health_profile_wh)} แถว")
display(health_profile_wh.head())

✅ Health Profile Warehouse: 100 แถว


,member_id,จำนวนครั้งการตรวจ,เพศ,อายุ,ตำบล_id,ตำบล,อำเภอ_id,อำเภอ,จังหวัด_id,จังหวัด,เขต,กลุ่มอายุ,ประวัติการเจ็บป่วย_code,ประวัติการเจ็บป่วย,ประวัติแพ้ยา_code,ประวัติแพ้ยา,ประวัติแพ้อาหาร_code,ประวัติแพ้อาหาร,ญาติสายตรงNCDs_code,ญาติสายตรงNCDs
0,10000036,1,ชาย,50,500101,ต. ศรีภูมิ,5001.0,อ. เมืองเชียงใหม่,50.0,จ. เชียงใหม่,2.0,อายุระหว่าง 35-59 ปี,-1,ไม่มีประวัติ,1,มีประวัติ,-1,ไม่มีประวัติ,-1,ไม่มีประวัติ
1,10000058,2,หญิง,44,120102,ต. ตลาดขวัญ,1201.0,อ. เมืองนนทบุรี,12.0,จ. นนทบุรี,6.0,อายุระหว่าง 35-59 ปี,-1,ไม่มีประวัติ,-1,ไม่มีประวัติ,-1,ไม่มีประวัติ,-1,ไม่มีประวัติ
2,10000025,1,หญิง,73,400101,ต. ในเมือง,4001.0,อ. เมืองขอนแก่น,40.0,จ. ขอนแก่น,1.0,อายุ 60 ปีขึ้นไป,-1,ไม่มีประวัติ,1,มีประวัติ,-1,ไม่มีประวัติ,-1,ไม่มีประวัติ
3,10000040,1,หญิง,48,500101,ต. ศรีภูมิ,5001.0,อ. เมืองเชียงใหม่,50.0,จ. เชียงใหม่,2.0,อายุระหว่าง 35-59 ปี,-1,ไม่มีประวัติ,-1,ไม่มีประวัติ,-1,ไม่มีประวัติ,1,มีประวัติ
4,10000076,1,หญิง,62,100101,แขวง พระบรมมหาราชวัง,1001.0,เขต พระนคร,10.0,กรุงเทพมหานคร,4.0,อายุ 60 ปีขึ้นไป,-1,ไม่มีประวัติ,-1,ไม่มีประวัติ,-1,ไม่มีประวัติ,-1,ไม่มีประวัติ


### Exercise 3.2: NCDs Disease Warehouse

**โจทย์**: สร้างตารางประวัติโรค NCDs

**ขั้นตอน**:
1. ดึงข้อมูลล่าสุดจาก `ncds_record`
2. สำหรับแต่ละสมาชิก สร้างรายการโรคของผู้ตรวจ (8 โรค)
3. สร้างรายการโรคของญาติ (9 โรค)
4. รวมเป็นตารางเดียว โดยแต่ละแถว = 1 โรค

**โรคของผู้ตรวจ**: cancer, chronic_kidney_disease, cirrhosis, dementia, diabetes, hyperlipidemia, hypertension, stroke

**โรคของญาติ**: cancer, cirrhosis, coronary_artery_disease, dementia, diabetes, hyperlipidemia, hypertension, stroke, obesity

In [26]:
# TODO: สร้าง NCDs Disease Warehouse
# Hint: ดูตัวอย่างจากไฟล์ detailed_health_warehouse_tutorial.ipynb

# เขียนโค้ดที่นี่
# ดึงข้อมูลล่าสุด
latest_records = ncds_record.sort_values('dateadd').groupby('memberid').tail(1)

# Mapping โรค
personal_diseases = {
    'record_cancer': 'โรคมะเร็ง',
    'record_chronic_kidney_disease': 'โรคไตเรื้อรัง',
    'record_cirrhosis': 'โรคตับแข็ง',
    'record_dementia': 'โรคสมองเสื่อม',
    'record_diabetes': 'โรคเบาหวาน',
    'record_hyperlipidemia': 'โรคไขมันในเลือด',
    'record_hypertension': 'โรคความดันโลหิตสูง',
    'record_stroke': 'โรคหลอดเลือดสมอง'
}

relatives_diseases = {
    'relatives_cancer': 'โรคมะเร็ง',
    'relatives_cirrhosis': 'โรคตับแข็ง',
    'relatives_coronary_artery_disease': 'โรคหลอดเลือดหัวใจ',
    'relatives_dementia': 'โรคสมองเสื่อม',
    'relatives_diabetes': 'โรคเบาหวาน',
    'relatives_hyperlipidemia': 'โรคไขมันในเลือด',
    'relatives_hypertension': 'โรคความดันโลหิตสูง',
    'relatives_stroke': 'โรคหลอดเลือดสมอง',
    'relatives_obesity': 'โรคอ้วนลงพุง'
}

# สร้างรายการโรค
disease_records = []
for _, row in latest_records.iterrows():
    member_id = row['memberid']

    # โรคของผู้ตรวจ
    p_diseases = [{'code': k, 'name': v} for k, v in personal_diseases.items()
                  if k in row and pd.notna(row[k]) and row[k] == 1]

    # โรคของญาติ
    r_diseases = [{'code': k, 'name': v} for k, v in relatives_diseases.items()
                  if k in row and pd.notna(row[k]) and row[k] == 1]

    max_len = max(len(p_diseases), len(r_diseases), 1)

    for i in range(max_len):
        disease_records.append({
            'member_id': member_id,
            'no': i + 1 if i < len(p_diseases) else 999,
            'โรค_code': p_diseases[i]['code'] if i < len(p_diseases) else 999,
            'โรคของผู้ตรวจ': p_diseases[i]['name'] if i < len(p_diseases) else 999,
            'no_โรคญาติผู้ตรวจ': i + 1 if i < len(r_diseases) else 999,
            'โรคญาติผู้ตรวจ_code': r_diseases[i]['code'] if i < len(r_diseases) else 999,
            'โรคของญาติผู้ตรวจ': r_diseases[i]['name'] if i < len(r_diseases) else 999
        })

ncds_disease_wh = pd.DataFrame(disease_records)
print(f"✅ NCDs Disease Warehouse: {len(ncds_disease_wh)} แถว")
ncds_disease_wh.head(10)


✅ NCDs Disease Warehouse: 308 แถว


,member_id,no,โรค_code,โรคของผู้ตรวจ,no_โรคญาติผู้ตรวจ,โรคญาติผู้ตรวจ_code,โรคของญาติผู้ตรวจ
0,10000036,999,999,999,1,relatives_coronary_artery_disease,โรคหลอดเลือดหัวใจ
1,10000036,999,999,999,2,relatives_dementia,โรคสมองเสื่อม
2,10000036,999,999,999,3,relatives_stroke,โรคหลอดเลือดสมอง
3,10000058,1,record_cancer,โรคมะเร็ง,1,relatives_diabetes,โรคเบาหวาน
4,10000058,2,record_dementia,โรคสมองเสื่อม,999,999,999
5,10000058,3,record_hypertension,โรคความดันโลหิตสูง,999,999,999
6,10000058,4,record_stroke,โรคหลอดเลือดสมอง,999,999,999
7,10000025,1,record_cirrhosis,โรคตับแข็ง,1,relatives_coronary_artery_disease,โรคหลอดเลือดหัวใจ
8,10000025,2,record_hypertension,โรคความดันโลหิตสูง,2,relatives_dementia,โรคสมองเสื่อม
9,10000025,999,999,999,3,relatives_obesity,โรคอ้วนลงพุง


### Exercise 3.3: Eating Behavior Warehouse

**โจทย์**: สร้างตารางพฤติกรรมการกิน

**ข้อมูลที่ต้องแปลง**:
- กินผัก (vegetable): 0→ไม่ทำเลย, 1→1-3 วัน, 2→4-6 วัน, 3→ทุกวัน
- กินขนมหวาน (dessert): 0→ทุกวัน, 1→4-6 วัน, 2→1-3 วัน, 3→ไม่ทำเลย (กลับด้าน!)
- ฯลฯ

In [30]:
# ดึงข้อมูลล่าสุด
latest_habits = ncds_habit.sort_values('dateadd').groupby('memberid').tail(1)

# ฟังก์ชันแปลงค่า
def map_score(score, mapping, reverse=False):
    if pd.isna(score) or score == -1: return 999
    score = int(score)
    return mapping.get(min(score, len(mapping)-1), mapping[len(mapping)-1])

# Mappings
freq_map = {0: "ไม่ทำเลย", 1: "1-3 วันต่อสัปดาห์", 2: "4-6 วันต่อสัปดาห์", 3: "ทุกวัน"}
freq_reverse = {0: "ทุกวัน", 1: "4-6 วันต่อสัปดาห์", 2: "1-3 วันต่อสัปดาห์", 3: "ไม่ทำเลย"}
add_map = {0: "เติมทุกครั้ง", 1: "เติมบางครั้ง", 2: "ไม่เติมเลย"}

# สร้าง Warehouse
eating_behavior_wh = pd.DataFrame({
    'date': pd.to_datetime(latest_habits['dateadd']).dt.date,
    'member_id': latest_habits['memberid'],
    'กินผัก_score': latest_habits['vegetable'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'กินผัก': latest_habits['vegetable'].apply(lambda x: map_score(x, freq_map)),
    'กินขนมหวาน_score': latest_habits['dessert'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'กินขนมหวาน': latest_habits['dessert'].apply(lambda x: map_score(x, freq_reverse)),
    'ดื่มเครื่องดื่มรสหวาน_score': latest_habits['sweet_drink'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'ดื่มเครื่องดื่มรสหวาน': latest_habits['sweet_drink'].apply(lambda x: map_score(x, freq_reverse)),
    'เติมน้ำตาลน้ำเชื่อม_score': latest_habits['sweet_food'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'เติมน้ำตาลน้ำเชื่อม': latest_habits['sweet_food'].apply(lambda x: map_score(x, add_map)),
    'อาหารไขมันสูง_score': latest_habits['fat_food'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'อาหารไขมันสูง': latest_habits['fat_food'].apply(lambda x: map_score(x, freq_reverse)),
    'กินอาหารแปรรูป_score': latest_habits['processed_food'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'กินอาหารแปรรูป': latest_habits['processed_food'].apply(lambda x: map_score(x, freq_reverse)),
    'กินอาหารสำเร็จรูป_score': latest_habits['instant_food'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'กินอาหารสำเร็จรูป': latest_habits['instant_food'].apply(lambda x: map_score(x, freq_reverse)),
    'กินอาหารปรุงรสด้วยผงปรุงรส_score': latest_habits['gourmet_powder'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'กินอาหารปรุงรสด้วยผงปรุงรส': latest_habits['gourmet_powder'].apply(lambda x: map_score(x, freq_reverse)),
    'เติมน้ำปลา_score': latest_habits['fish_sauce'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'เติมน้ำปลา': latest_habits['fish_sauce'].apply(lambda x: map_score(x, add_map))
})

print(f"✅ Eating Behavior Warehouse: {len(eating_behavior_wh)} แถว")
eating_behavior_wh.head()

✅ Eating Behavior Warehouse: 100 แถว


,date,member_id,กินผัก_score,กินผัก,กินขนมหวาน_score,กินขนมหวาน,ดื่มเครื่องดื่มรสหวาน_score,ดื่มเครื่องดื่มรสหวาน,เติมน้ำตาลน้ำเชื่อม_score,เติมน้ำตาลน้ำเชื่อม,อาหารไขมันสูง_score,อาหารไขมันสูง,กินอาหารแปรรูป_score,กินอาหารแปรรูป,กินอาหารสำเร็จรูป_score,กินอาหารสำเร็จรูป,กินอาหารปรุงรสด้วยผงปรุงรส_score,กินอาหารปรุงรสด้วยผงปรุงรส,เติมน้ำปลา_score,เติมน้ำปลา
76,2025-06-02,10000036,3,ทุกวัน,0,ทุกวัน,3,ไม่ทำเลย,1,เติมบางครั้ง,1,4-6 วันต่อสัปดาห์,2,1-3 วันต่อสัปดาห์,0,ทุกวัน,1,4-6 วันต่อสัปดาห์,2,ไม่เติมเลย
113,2025-06-10,10000058,999,999,3,ไม่ทำเลย,2,1-3 วันต่อสัปดาห์,0,เติมทุกครั้ง,0,ทุกวัน,0,ทุกวัน,3,ไม่ทำเลย,3,ไม่ทำเลย,0,เติมทุกครั้ง
53,2025-06-12,10000025,1,1-3 วันต่อสัปดาห์,0,ทุกวัน,0,ทุกวัน,1,เติมบางครั้ง,3,ไม่ทำเลย,1,4-6 วันต่อสัปดาห์,0,ทุกวัน,0,ทุกวัน,2,ไม่เติมเลย
81,2025-06-12,10000040,2,4-6 วันต่อสัปดาห์,3,ไม่ทำเลย,999,999,2,ไม่เติมเลย,1,4-6 วันต่อสัปดาห์,3,ไม่ทำเลย,3,ไม่ทำเลย,2,1-3 วันต่อสัปดาห์,2,ไม่เติมเลย
141,2025-06-13,10000076,0,ไม่ทำเลย,999,999,999,999,1,เติมบางครั้ง,2,1-3 วันต่อสัปดาห์,1,4-6 วันต่อสัปดาห์,2,1-3 วันต่อสัปดาห์,3,ไม่ทำเลย,1,เติมบางครั้ง


### Exercise 3.4: Comprehensive Behavior Warehouse

**โจทย์**: สร้างตารางพฤติกรรมรวม

**ข้อมูลที่ต้องแปลง**:
- การเคลื่อนไหว (movement)
- การสูบบุหรี่ (smoking)
- การนอนหลับ (sleep)
- ภาวะซึมเศร้า (2q_1)
- ความเครียด (2q_2)
- การดื่มแอลกอฮอล์ (alcohol)
- การออกกำลังกาย (exercise)

In [31]:
# TODO: สร้าง Comprehensive Behavior Warehouse
# Hint: คล้ายกับ Eating Behavior แต่เป็นพฤติกรรมด้านอื่น

# เขียนโค้ดที่นี่
# Mappings
movement_map = {0: "ไม่ทำเลย", 1: "1-2 วันต่อสัปดาห์", 2: "มากกว่าหรือเท่ากับ 3 วันต่อสัปดาห์"}
smoking_map = {-1: "ไม่สูบ", 1: "สูบ"}
sleep_map = {0: "0 - 2 วันต่อสัปดาห์", 1: "3 - 5 วันต่อสัปดาห์", 2: "6 - 7 วันต่อสัปดาห์"}
depression_map = {0: "ไม่มีความเสี่ยง", 1: "มีความเสี่ยง"}
stress_map = {0: "แทบไม่มี", 1: "เป็นบางครั้ง", 2: "บ่อยครั้ง", 3: "เป็นประจำ"}
alcohol_map = {0: "ไม่ดื่มเลย", 1: "ดื่ม 1 - 3 วันต่อสัปดาห์", 2: "ดื่ม 4 - 6 วันต่อสัปดาห์", 3: "ดื่มเป็นประจำทุกวัน"}
exercise_map = {
    0: "ไม่ออกกำลังกายหรือออกกำลังกายน้อยมาก",
    1: "ออกกำลังกายน้อย เล่นกีฬา 1-3 วัน/สัปดาห์",
    2: "ออกกำลังกายปานกลาง เล่นกีฬา 3-5 วัน/สัปดาห์",
    3: "ออกกำลังกายหนัก เล่นกีฬา 6-7 วัน/สัปดาห์",
    4: "ออกกำลังกายหนักมากเป็นนักกีฬา"
}

# สร้าง Warehouse
comprehensive_behavior_wh = pd.DataFrame({
    'date': pd.to_datetime(latest_habits['dateadd']).dt.date,
    'member_id': latest_habits['memberid'],
    'การเคลื่อนไหวร่างกาย_score': latest_habits['movement'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'การเคลื่อนไหวร่างกาย': latest_habits['movement'].apply(lambda x: map_score(x, movement_map)),
    'สูบบุหรี่_code': latest_habits['smoking'].apply(lambda x: 999 if pd.isna(x) else int(x)),
    'สูบบุหรี่': latest_habits['smoking'].apply(lambda x: smoking_map.get(int(x), 999) if pd.notna(x) else 999),
    'การนอนหลับ_score': latest_habits['sleep'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'การนอนหลับ': latest_habits['sleep'].apply(lambda x: map_score(x, sleep_map)),
    'ภาวะซึมเศร้า_score': latest_habits['2q_1'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'ภาวะซึมเศร้า': latest_habits['2q_1'].apply(lambda x: map_score(x, depression_map)),
    'ความเครียด_score': latest_habits['2q_2'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'ความเครียด': latest_habits['2q_2'].apply(lambda x: map_score(x, stress_map)),
    'การดื่มแอล_score': latest_habits['alcohol'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'การดื่มแอล': latest_habits['alcohol'].apply(lambda x: map_score(x, alcohol_map)),
    'ออกกำลังกาย_score': latest_habits['exercise'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'ออกกำลังกาย': latest_habits['exercise'].apply(lambda x: map_score(x, exercise_map))
})

print(f"✅ Comprehensive Behavior Warehouse: {len(comprehensive_behavior_wh)} แถว")
comprehensive_behavior_wh.head()


✅ Comprehensive Behavior Warehouse: 100 แถว


,date,member_id,การเคลื่อนไหวร่างกาย_score,การเคลื่อนไหวร่างกาย,สูบบุหรี่_code,สูบบุหรี่,การนอนหลับ_score,การนอนหลับ,ภาวะซึมเศร้า_score,ภาวะซึมเศร้า,ความเครียด_score,ความเครียด,การดื่มแอล_score,การดื่มแอล,ออกกำลังกาย_score,ออกกำลังกาย
76,2025-06-02,10000036,2,มากกว่าหรือเท่ากับ 3 วันต่อสัปดาห์,-1,ไม่สูบ,999,999,0,ไม่มีความเสี่ยง,0,แทบไม่มี,3,ดื่มเป็นประจำทุกวัน,0,ไม่ออกกำลังกายหรือออกกำลังกายน้อยมาก
113,2025-06-10,10000058,2,มากกว่าหรือเท่ากับ 3 วันต่อสัปดาห์,0,999,1,3 - 5 วันต่อสัปดาห์,1,มีความเสี่ยง,1,เป็นบางครั้ง,3,ดื่มเป็นประจำทุกวัน,1,ออกกำลังกายน้อย เล่นกีฬา 1-3 วัน/สัปดาห์
53,2025-06-12,10000025,1,1-2 วันต่อสัปดาห์,-1,ไม่สูบ,1,3 - 5 วันต่อสัปดาห์,1,มีความเสี่ยง,999,999,0,ไม่ดื่มเลย,1,ออกกำลังกายน้อย เล่นกีฬา 1-3 วัน/สัปดาห์
81,2025-06-12,10000040,2,มากกว่าหรือเท่ากับ 3 วันต่อสัปดาห์,0,999,1,3 - 5 วันต่อสัปดาห์,0,ไม่มีความเสี่ยง,0,แทบไม่มี,1,ดื่ม 1 - 3 วันต่อสัปดาห์,2,ออกกำลังกายปานกลาง เล่นกีฬา 3-5 วัน/สัปดาห์
141,2025-06-13,10000076,1,1-2 วันต่อสัปดาห์,0,999,2,6 - 7 วันต่อสัปดาห์,1,มีความเสี่ยง,0,แทบไม่มี,2,ดื่ม 4 - 6 วันต่อสัปดาห์,0,ไม่ออกกำลังกายหรือออกกำลังกายน้อยมาก


In [45]:
# TODO: สร้าง Disease Risk Warehouse
# Hint: 1. ดึงค่าจาก memberdata (sys/dias, bmi, waistline, height)
#       2. สร้างฟังก์ชันคำนวณความเสี่ยงทั้ง 3 ประเภท
#       3. Loop แต่ละสมาชิกและคำนวณ
#       4. สร้าง DataFrame จากผลลัพธ์

# เขียนโค้ดที่นี่

# 1. ดึงค่าจาก memberdata
latest_memberdata = memberdata.sort_values('dateadd').groupby('memberid').tail(1)

def get_member_data(member_id, name):
    """Helper function to get a specific value for a member"""
    value_row = latest_memberdata[(latest_memberdata['memberid'] == member_id) & (latest_memberdata['name'] == name)]
    if not value_row.empty:
        # Attempt to convert to numeric, handle errors by returning original value
        try:
            return pd.to_numeric(value_row['value'].iloc[0])
        except (ValueError, TypeError):
            return value_row['value'].iloc[0]
    return None

# 2. สร้างฟังก์ชันคำนวณความเสี่ยง

def calculate_blood_pressure_risk(sys, dias):
    """Calculate Blood Pressure Risk"""
    if sys is None or dias is None:
        return 'ไม่ทราบ', None

    try:
        sys = int(sys)
        dias = int(dias)
        if sys < 120 and dias < 80:
            return 'ปกติ', f'{sys}/{dias}'
        elif (sys >= 120 and sys <= 129) and dias < 80:
             return 'ความดันโลหิตสูงเล็กน้อย', f'{sys}/{dias}'
        elif (sys >= 130 and sys <= 139) or (dias >= 80 and dias <= 89):
             return 'ความดันโลหิตสูงระยะที่ 1', f'{sys}/{dias}'
        elif sys >= 140 or dias >= 90:
             return 'ความดันโลหิตสูงระยะที่ 2', f'{sys}/{dias}'
        elif sys >= 180 or dias >= 120:
             return 'ความดันโลหิตสูงวิกฤต', f'{sys}/{dias}'
        else:
             return 'ไม่ทราบ', f'{sys}/{dias}'
    except (ValueError, TypeError):
        return 'ข้อมูลไม่ถูกต้อง', f'{sys}/{dias}'


def calculate_findrisc(age, bmi, waistline, sys_dias, relatives_diabetes):
    """Calculate FINDRISC score"""
    score = 0
    if age is not None:
        if 45 <= age <= 54: score += 2
        elif 55 <= age <= 64: score += 3
        elif age >= 65: score += 4

    if bmi is not None:
        if 25 <= bmi <= 30: score += 1
        elif bmi > 30: score += 3

    if waistline is not None:
        # Assuming female/male based on typical ranges, adjust if gender data available
        # Using thresholds for Asian populations (example)
        if isinstance(waistline, (int, float)):
            if waistline >= 80 and waistline < 90: score += 3 # Female
            elif waistline >= 90: score += 4 # Male

    if sys_dias is not None and isinstance(sys_dias, str):
        try:
            sys, dias = map(int, sys_dias.split('/'))
            if sys >= 140 or dias >= 90: score += 2
        except ValueError:
            pass # Handle cases where sys_dias is not in expected format

    if relatives_diabetes is not None:
         if relatives_diabetes == 1: score += 5 # Assuming 1 means history

    risk_level = 'ไม่ทราบ'
    if 0 <= score <= 7: risk_level = 'ต่ำ'
    elif 7 < score <= 11: risk_level = 'ค่อนข้างต่ำ'
    elif 11 < score <= 14: risk_level = 'ปานกลาง'
    elif 14 < score <= 20: risk_level = 'สูง'
    elif score > 20: risk_level = 'สูงมาก'

    return risk_level, f'{score}/{risk_level}' # Return score and level for display


def calculate_cv_risk(age, gender, smoking, diabetes, sys, waistline, height):
    """Calculate Cardiovascular Risk (Simplified Example)"""
    # This is a simplified example, real CV risk models are more complex
    # Replace with a proper model if needed (e.g., Thai CV Risk Score)
    if age is None or gender is None or smoking is None or diabetes is None or sys is None or waistline is None or height is None:
         return 'ไม่ทราบ', None

    try:
        score = 0
        # Basic scoring based on risk factors
        if age >= 40: score += (age - 40) * 0.2 # Age contribution (example)
        if gender == 1: score += 2 # Male (example)
        if smoking == 1: score += 3 # Smoker (example)
        if diabetes == 1: score += 4 # Diabetes (example)
        if sys >= 140: score += (sys - 140) * 0.1 # High BP (example)
        if isinstance(waistline, (int, float)) and isinstance(height, (int, float)) and height > 0:
             whr = waistline / height
             if whr > 0.5: score += (whr - 0.5) * 10 # High Waist-to-Height Ratio (example)


        # Convert score to a risk percentage or category (simplified)
        # This mapping is completely illustrative and NOT medically accurate
        risk_pct = min(score * 2, 100) # Example conversion to percentage

        risk_level = 'ไม่ทราบ'
        if risk_pct < 5: risk_level = 'ต่ำ'
        elif 5 <= risk_pct < 15: risk_level = 'ปานกลาง'
        elif 15 <= risk_pct < 30: risk_level = 'สูง'
        elif risk_pct >= 30: risk_level = 'สูงมาก'

        return risk_level, f'{risk_pct:.2f}%'
    except (ValueError, TypeError):
        return 'ข้อมูลไม่ถูกต้อง', None


# 3. Loop แต่ละสมาชิกและคำนวณ
disease_risk_records = []

for member_id in latest_memberdata['memberid'].unique():
    age = get_member_data(member_id, 'age')
    gender = get_member_data(member_id, 'gender')
    bmi = get_member_data(member_id, 'bmi')
    waistline = get_member_data(member_id, 'waistline')
    sys_dias = get_member_data(member_id, 'sys/dias')
    smoking = get_member_data(member_id, 'smoking') # Assuming smoking data exists in memberdata or can be joined
    diabetes = get_member_data(member_id, 'record_diabetes') # Assuming diabetes history exists in memberdata or can be joined from records
    relatives_diabetes = get_member_data(member_id, 'relatives_diabetes') # Assuming data exists or can be joined from records
    height = get_member_data(member_id, 'height')
    sys = get_member_data(member_id, 'sys') # Assuming sys is available separately or extracted from sys/dias


    # Calculate Blood Pressure Risk
    bp_sys, bp_dias = None, None
    if isinstance(sys_dias, str) and '/' in sys_dias:
         try:
              bp_sys, bp_dias = map(int, sys_dias.split('/'))
         except ValueError:
              pass

    bp_level, bp_score_display = calculate_blood_pressure_risk(bp_sys, bp_dias)
    disease_risk_records.append({
        'member_id': member_id,
        'ภาวะเสี่ยง': 'โรคความดันโลหิตสูง',
        'ระดับความเสี่ยง': bp_level,
        'คะแนน': bp_score_display
    })

    # Calculate FINDRISC
    # Need to get relatives_diabetes from latest_records, not memberdata
    latest_rec_for_member = latest_records[latest_records['memberid'] == member_id]
    rel_dm = latest_rec_for_member['relatives_diabetes'].iloc[0] if not latest_rec_for_member.empty and 'relatives_diabetes' in latest_rec_for_member.columns else None


    dm_level, dm_score_display = calculate_findrisc(age, bmi, waistline, sys_dias, rel_dm)
    disease_risk_records.append({
        'member_id': member_id,
        'ภาวะเสี่ยง': 'เสี่ยงโรคเบาหวาน',
        'ระดับความเสี่ยง': dm_level,
        'คะแนน': dm_score_display
    })

    # Calculate Cardiovascular Risk
    # Need to get smoking and record_diabetes from latest_records
    smoking_status = latest_rec_for_member['smoking'].iloc[0] if not latest_rec_for_member.empty and 'smoking' in latest_rec_for_member.columns else None
    rec_dm = latest_rec_for_member['record_diabetes'].iloc[0] if not latest_rec_for_member.empty and 'record_diabetes' in latest_rec_for_member.columns else None

    # Extract sys from sys_dias string for CV risk calculation if not available separately
    cv_sys = bp_sys

    cv_level, cv_score_display = calculate_cv_risk(age, gender, smoking_status, rec_dm, cv_sys, waistline, height)
    disease_risk_records.append({
        'member_id': member_id,
        'ภาวะเสี่ยง': 'เสี่ยงโรคหัวใจและหลอดเลือด',
        'ระดับความเสี่ยง': cv_level,
        'คะแนน': cv_score_display
    })


# 4. สร้าง DataFrame จากผลลัพธ์
disease_risk_wh = pd.DataFrame(disease_risk_records)

print(f"✅ Disease Risk Warehouse: {len(disease_risk_wh)} แถว")
display(disease_risk_wh.head())

✅ Disease Risk Warehouse: 300 แถว


,member_id,ภาวะเสี่ยง,ระดับความเสี่ยง,คะแนน
0,10000036,โรคความดันโลหิตสูง,ความดันโลหิตสูงระยะที่ 2,157/95
1,10000036,เสี่ยงโรคเบาหวาน,ต่ำ,2/ต่ำ
2,10000036,เสี่ยงโรคหัวใจและหลอดเลือด,ไม่ทราบ,None
3,10000058,โรคความดันโลหิตสูง,ไม่ทราบ,None
4,10000058,เสี่ยงโรคเบาหวาน,ต่ำ,5/ต่ำ


### Exercise 3.5: Disease Risk Warehouse

**โจทย์**: สร้างตารางความเสี่ยงโรค 3 ประเภท

**ความเสี่ยงที่ต้องคำนวณ**:
1. **โรคความดันโลหิตสูง**: จากค่า sys/dias
2. **เสี่ยงโรคเบาหวาน**: ใช้ FINDRISC algorithm (คำนวณจาก age, gender, BMI, waistline, sys/dias, relatives_diabetes)
3. **เสี่ยงโรคหัวใจและหลอดเลือด**: คำนวณจาก age, gender, smoking, diabetes, sys, waist/height ratio

**โครงสร้างข้อมูล**:
- แต่ละสมาชิกจะมี 3 แถว (1 แถวต่อ 1 ประเภทความเสี่ยง)
- คอลัมน์: member_id, ภาวะเสี่ยง, ระดับความเสี่ยง, คะแนน

**Hint**: ดูตัวอย่างจาก detailed_health_warehouse_tutorial.ipynb

---

## 📤 Part 4: อัปโหลดไปยัง Google Sheets (15 นาที)

### Exercise 4.1: Authentication

**โจทย์**: Authenticate กับ Google เพื่อใช้งาน Google Sheets API

In [38]:
 #TODO: Authenticate กับ Google
# Hint: ใช้ from google.colab import auth
#       auth.authenticate_user()
#       แล้ว authorize กับ gspread

In [39]:
from google.colab import auth
auth.authenticate_user()

### Exercise 4.3: อัปโหลดข้อมูล

**โจทย์**: อัปโหลดข้อมูลทั้ง 5 warehouses ไปยัง Google Sheets โดยแต่ละ warehouse เป็น 1 sheet

**ชื่อ Sheet**:
- 01-ข้อมูลผู้ตรวจสุขภาพ
- 02-ประวัติโรคNCDs
- 03-พฤติกรรมการกิน
- 04-พฤติกรรมรวม
- 05-ความเสี่ยงโรค

In [46]:
# TODO: อัปโหลดข้อมูลทั้ง 5 warehouses ไปยัง Google Sheets โดยแต่ละ warehouse เป็น 1 sheet

# เขียนโค้ดที่นี่

# Authorize gspread using the OAuth credentials
import google.auth
from gspread import Client

# Get the credentials from the authenticated user
credentials, project = google.auth.default()

# Create a gspread client using the credentials
gc = Client(credentials)


# Open the spreadsheet - replace 'YOUR_SPREADSHEET_NAME' with your desired name
# You might need to create a new spreadsheet manually or use gspread to create one
spreadsheet = gc.create('Health Data Warehouse (Student Version)') # Replace with your desired name

# Define the list of warehouses and their sheet names
warehouses_to_upload = [
    (health_profile_wh, '01-ข้อมูลผู้ตรวจสุขภาพ'),
    (ncds_disease_wh, '02-ประวัติโรคNCDs'),
    (eating_behavior_wh, '03-พฤติกรรมการกิน'),
    (comprehensive_behavior_wh, '04-พฤติกรรมรวม'),
    (disease_risk_wh, '05-ความเสี่ยงโรค')
]

print(f"📤 กำลังอัปโหลดข้อมูลไปยัง Google Sheets: '{spreadsheet.title}'")

for df, sheet_name in warehouses_to_upload:
    upload_to_sheet(spreadsheet, df, sheet_name)
    print(f"✅ อัปโหลด '{sheet_name}' สำเร็จ")

print("✅ อัปโหลดข้อมูลทั้งหมดสำเร็จ")
print(f"Google Sheets URL: {spreadsheet.url}")

📤 กำลังอัปโหลดข้อมูลไปยัง Google Sheets: 'Health Data Warehouse (Student Version)'
✅ อัปโหลด '01-ข้อมูลผู้ตรวจสุขภาพ' สำเร็จ
✅ อัปโหลด '02-ประวัติโรคNCDs' สำเร็จ
✅ อัปโหลด '03-พฤติกรรมการกิน' สำเร็จ
✅ อัปโหลด '04-พฤติกรรมรวม' สำเร็จ
✅ อัปโหลด '05-ความเสี่ยงโรค' สำเร็จ
✅ อัปโหลดข้อมูลทั้งหมดสำเร็จ
Google Sheets URL: https://docs.google.com/spreadsheets/d/1qRhISCv50Alr0FfwI7sLq5Jt2jYXG5RyzCZL_sG7m04


In [44]:
def upload_to_sheet(spreadsheet, df, sheet_name):
    """อัปโหลด DataFrame ไปยัง Google Sheet"""
    # TODO: เขียนโค้ดอัปโหลด
    # Hint: 1. หา/สร้าง worksheet
    #       2. ล้างข้อมูลเดิม (worksheet.clear())
    #       3. แปลง DataFrame เป็น list (ระวัง date objects!)
    #       4. อัปโหลด (worksheet.update())

    # เขียนโค้ดที่นี่
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
    except gspread.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows="100", cols="20")

    worksheet.clear()

    # Convert DataFrame to a list of lists, handling potential date/datetime objects
    # Ensure all data types are compatible with Google Sheets (strings, numbers)
    data_to_upload = [df.columns.tolist()] + df.astype(str).values.tolist()

    worksheet.update(data_to_upload)

# TODO: อัปโหลดทั้ง 5 warehouses
# เขียนโค้ดที่นี่

---

## 📝 Part 5: สรุปและคำถาม (10 นาที)

### คำถามท้ายบท:

**Q1**: ทำไมต้องดึงข้อมูล "ล่าสุด" ของแต่ละสมาชิก ไม่ใช่ข้อมูลทั้งหมด?

**คำตอบ**: _______________________________________________________

---

**Q2**: ในตาราง NCDs Disease ทำไมถึงมีหลายแถวต่อ 1 สมาชิก?

**คำตอบ**: _______________________________________________________

---

**Q3**: อะไรคือความแตกต่างระหว่าง "_score" และ "label" ในตาราง Eating Behavior?

**คำตอบ**: _______________________________________________________

---

**Q4**: ลองคิดว่าถ้าจะเพิ่ม Warehouse ตารางที่ 6 ควรจะเป็นตารางอะไร? และมีข้อมูลอะไรบ้าง?

**คำตอบ**: _______________________________________________________

---

## ✅ Checklist สำหรับการส่งงาน

- [ ] สร้าง Health Profile Warehouse ได้อย่างน้อย 100 แถว
- [ ] สร้าง NCDs Disease Warehouse ได้
- [ ] สร้าง Eating Behavior Warehouse ได้
- [ ] สร้าง Comprehensive Behavior Warehouse ได้
- [ ] สร้าง Disease Risk Warehouse ได้
- [ ] อัปโหลดข้อมูลไปยัง Google Sheets สำเร็จ
- [ ] ตอบคำถามท้ายบทครบทั้ง 4 ข้อ
- [ ] แชร์ Google Sheets ให้อาจารย์ดูได้

## 📤 วิธีการส่งงาน

1. Save notebook นี้ (Download .ipynb)
2. Copy URL ของ Google Sheets ที่สร้าง
3. ส่งทั้ง 2 อย่างใน Google Classroom

---

**Google Sheets URL**: _______________________________________________________

**หมายเหตุ**: กรุณาแชร์ Google Sheets ให้อาจารย์สามารถดูได้ (Viewer)

---

## 🎓 เกณฑ์การให้คะแนน (100 คะแนน)

| หัวข้อ | คะแนน |
|--------|-------|
| Part 1-2: เตรียมสภาพแวดล้อมและดึงข้อมูล | 15 |
| Part 3.1: Health Profile Warehouse | 20 |
| Part 3.2: NCDs Disease Warehouse | 12 |
| Part 3.3: Eating Behavior Warehouse | 12 |
| Part 3.4: Comprehensive Behavior Warehouse | 12 |
| Part 3.5: Disease Risk Warehouse | 19 |
| Part 4: อัปโหลดไปยัง Google Sheets | 5 |
| Part 5: คำถามท้ายบท | 5 |
| **รวม** | **100** |

---

**สิ่งที่ได้เรียนรู้**:
✅ การทำ Data Transformation  
✅ การใช้ Pandas สำหรับ Data Wrangling  
✅ การสร้าง Data Warehouse  
✅ การคำนวณความเสี่ยงโรค (FINDRISC, Cardiovascular Risk)  
✅ การใช้ Google Sheets API  

**Good Luck! 🎓**

In [ ]:
# ปิดการเชื่อมต่อ Database
conn.close()
print("✅ ปิดการเชื่อมต่อ Database เรียบร้อยแล้ว")